In [1]:
import gi
from gi.repository import GdkPixbuf

gi.require_version("Gtk", "3.0")
from gi.repository import Gtk
import cv2
import os
import json

class im_wiz_win(Gtk.Window):
    def __init__(self, preset):
        self.img_iter = 0
        while onlyfiles[self.img_iter] in data:
            self.img_iter +=1
            if self.img_iter >= len(onlyfiles):
                break
        Gtk.Window.__init__(self, title='Image Wizard')
        self.width = 550
        self.set_default_size(self.width, self.width)
        
        
        if self.img_iter < len(onlyfiles):
            self.img_name = onlyfiles[self.img_iter]
            self.props.title = self.img_name + ' ' + str(self.img_iter+1) + '/' + str(len(onlyfiles))
            self.place_options = preset["place_options"]
            self.id_options = preset["id_options"]

            self.box = Gtk.Box(spacing=6)
            self.add(self.box)
            data[self.img_name] = {"tags": preset["tags"],"position" : {}, "technic" : preset["technic"], "place": self.place_options[0]}

            # Image
            pix = GdkPixbuf.Pixbuf.new_from_file(path+self.img_name)
            self.img_height = pix.get_height()
            self.img_width = pix.get_width()
            pixbuf = GdkPixbuf.Pixbuf.new_from_file_at_scale(path+self.img_name, self.width, -1, True)
            self.height = pixbuf.get_height()
            self.resize(self.width, self.height)
            self.image = Gtk.Image()
            self.image.set_from_pixbuf(pixbuf)
            self.img_box = Gtk.EventBox()
            self.img_box.add(self.image)
            self.img_box.connect('button-press-event', self.onclick_position)

            self.box.pack_start(self.img_box, True, True, 0)

            ## Box in the right
            self.box_r = Gtk.Box(orientation=Gtk.Orientation.VERTICAL,spacing=6)

            # Dropdown local
            self.local_drop = Gtk.ComboBoxText()
            for option in self.place_options:
                self.local_drop.append_text(option)
            self.local_drop.set_active(0)
            self.local_drop.connect("changed", self.local_changed)

            self.box_r.pack_start(self.local_drop, True, True, 0)
            # Dropdown ids
            self.id_drop = Gtk.ComboBoxText()
            for option in self.id_options:
                self.id_drop.append_text(option)
            self.id_drop.set_active(0)
            self.id_drop.connect("changed", self.id_changed)

            self.box_r.pack_start(self.id_drop, True, True, 0)
            # Positions print
            self.positions_label = Gtk.Label(label="")

            self.box_r.pack_start(self.positions_label, True, True, 0)
            # Buttons
            self.box_buttons = Gtk.Box(spacing=6)

            # Button Delete
            self.btn_delete = Gtk.Button(label="Delete")
            self.btn_delete.connect("clicked", self.btn_delete_clicked)

            self.box_buttons.pack_start(self.btn_delete, True, True, 0)
            # Button Next
            self.btn_next = Gtk.Button(label="Next")
            self.btn_next.connect("clicked", self.btn_next_clicked)

            self.box_buttons.pack_start(self.btn_next, True, True, 0)

            self.box_r.pack_start(self.box_buttons, True, True, 0)
            self.box.pack_start(self.box_r, True, True, 0)
        else:
            self.add(Gtk.Label(label='All images were already marked'))
            
    def btn_next_clicked(self, widget):
        curr_local = self.local_drop.get_model()[self.local_drop.get_active()][0]
        while onlyfiles[self.img_iter] in data:
            self.img_iter +=1
            if self.img_iter >= len(onlyfiles):
                break
        if self.img_iter < len(onlyfiles):
            self.img_name = onlyfiles[self.img_iter]
            pix = GdkPixbuf.Pixbuf.new_from_file(path+self.img_name)
            self.img_height = pix.get_height()
            self.img_width = pix.get_width()
            pixbuf = GdkPixbuf.Pixbuf.new_from_file_at_scale(path+self.img_name, self.width, -1, True)
            self.height = pixbuf.get_height()
            self.image.set_from_pixbuf(pixbuf)
            self.resize(self.width, self.height)
            data[self.img_name] = {"tags": preset["tags"],"position" : {}, "technic" : preset["technic"], "place": curr_local}
            self.props.title = self.img_name  + ' ' + str(self.img_iter+1) + '/' + str(len(onlyfiles))
            self.update_positions()
        else:
            self.close()
    def btn_delete_clicked(self, widget):
        if os.path.exists(path+self.img_name):
            os.remove(path+self.img_name)
        else:
            print("The file does not exist")
            exit()
        del(onlyfiles[self.img_iter])
        del(data[self.img_name])
        if self.img_iter < len(onlyfiles):
            while onlyfiles[self.img_iter] in data:
                self.img_iter +=1
                if self.img_iter >= len(onlyfiles):
                    break
            if self.img_iter < len(onlyfiles):
                self.img_name = onlyfiles[self.img_iter]
                pix = GdkPixbuf.Pixbuf.new_from_file(path+self.img_name)
                self.img_height = pix.get_height()
                self.img_width = pix.get_width()
                pixbuf = GdkPixbuf.Pixbuf.new_from_file_at_scale(path+self.img_name, self.width, -1, True)
                self.height = pixbuf.get_height()
                self.image.set_from_pixbuf(pixbuf)
                self.resize(self.width, self.height)
                data[self.img_name] = {"tags": preset["tags"], "position" : {}, "technic" : preset["technic"], "place": self.place_options[0]}
                self.positions_label.props.label =  "File Deleted!"
                self.props.title = self.img_name  + ' ' + str(self.img_iter+1) + '/' + str(len(onlyfiles))
            else:
                self.close()
        else:
            self.close()
    def id_changed(self, combo):
        self.update_positions()
    def local_changed(self, combo):
        tree_iter = combo.get_active_iter()
        if tree_iter is not None:
            model = combo.get_model()
            curr_local = model[tree_iter][0]
            data[self.img_name]["place"] = curr_local
    def onclick_position (self,box , event):
        curr_id = self.id_drop.get_model()[self.id_drop.get_active()][0]
        x = int((event.x/self.width)*self.img_width)
        y = int((event.y/self.height)*self.img_height)
        
        if curr_id in data[self.img_name]['position']:
            data[self.img_name]['position'][curr_id].append([x,y])
        else:
            data[self.img_name]['position'][curr_id] = [[x,y]]
        self.update_positions()
    def update_positions(self):
        label_text = ""
        curr_id = self.id_drop.get_model()[self.id_drop.get_active()][0]
        if curr_id in data[self.img_name]['position']:
            for pos in data[self.img_name]['position'][curr_id]:
                label_text+='['+str(pos[0])+', '+str(pos[1])+']\n'
        self.positions_label.props.label =  label_text

try:
    with open('../algorithm_performance_data/real_positions/ArUco_v1.json', 'r') as outfile:
        data = json.load(outfile)
except:
    data = {}
        
preset = {"tags": ["v3"],
          "technic" : "colors",
          "id_options" : ["#0", "#1", "#2", "#3"], 
          "place_options" : ["daylight", "night right bulb", "night center bulb", "night left bulb"]}
path = '../img/ArUco/'
onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
onlyfiles.sort()

img_name = ''
win = im_wiz_win(preset)
win.connect("destroy", Gtk.main_quit)
win.show_all()
Gtk.main()

with open('../algorithm_performance_data/real_positions/ArUco_v1.json', 'w') as outfile:
    json.dump(data, outfile)

<ipython-input-1-1dc69eea3caf>:2: PyGIWarning: GdkPixbuf was imported without specifying a version first. Use gi.require_version('GdkPixbuf', '2.0') before import to ensure that the right version gets loaded.
  from gi.repository import GdkPixbuf


In [2]:
import cv2
import matplotlib.pyplot as plt
import os

# Rotate portrait to landscape orientation

path = '../img/ArUco/v1/'
onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
onlyfiles.sort()
for f in onlyfiles:
    im = cv2.imread(path+f)
    print(im.shape)
    if(im.shape[0]>im.shape[1]):
        im = cv2.rotate(im, cv2.ROTATE_90_CLOCKWISE)
#     plt.imshow(im)
    cv2.imwrite(path+f,im)

(4320, 5760, 3)
(4320, 5760, 3)
(4320, 5760, 3)
(4320, 5760, 3)
(4320, 5760, 3)
(4320, 5760, 3)
(4320, 5760, 3)
(4320, 5760, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(4320, 5760, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(4320, 5760, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(5760, 4320, 3)
(4320, 5760, 3)
(4320, 5760, 3)
(4320, 5

In [3]:
import gi
from gi.repository import GdkPixbuf

gi.require_version("Gtk", "3.0")
from gi.repository import Gtk
import cv2
import os
import json

class im_wiz_win(Gtk.Window):
    def __init__(self, preset):
        self.img_iter = 0
        #while onlyfiles[self.img_iter] in data:
        #    self.img_iter +=1
        #    if self.img_iter >= len(onlyfiles):
        #        break
        Gtk.Window.__init__(self, title='Image Wizard')
        self.width = 550
        self.set_default_size(self.width, self.width)
        
        
        if self.img_iter < len(onlyfiles):
            self.img_name = onlyfiles[self.img_iter]
            self.props.title = self.img_name + ' ' + str(self.img_iter+1) + '/' + str(len(onlyfiles))
            self.place_options = preset["place_options"]
            self.id_options = preset["id_options"]

            self.box = Gtk.Box(spacing=6)
            self.add(self.box)
            #data[self.img_name] = {"tags": preset["tags"],"position" : {}, "technic" : preset["technic"], "place": self.place_options[0]}

            # Image
            pix = GdkPixbuf.Pixbuf.new_from_file(path+self.img_name)
            self.img_height = pix.get_height()
            self.img_width = pix.get_width()
            pixbuf = GdkPixbuf.Pixbuf.new_from_file_at_scale(path+self.img_name, self.width, -1, True)
            self.height = pixbuf.get_height()
            self.image = Gtk.Image()
            self.image.set_from_pixbuf(pixbuf)
            self.img_box = Gtk.EventBox()
            self.img_box.add(self.image)
            self.img_box.connect('button-press-event', self.onclick_position)

            self.box.pack_start(self.img_box, True, True, 0)

            ## Box in the right
            self.box_r = Gtk.Box(orientation=Gtk.Orientation.VERTICAL,spacing=6)

            # Dropdown local
            self.local_drop = Gtk.ComboBoxText()
            for option in self.place_options:
                self.local_drop.append_text(option)
            self.local_drop.set_active(0)
            self.local_drop.connect("changed", self.local_changed)

            self.box_r.pack_start(self.local_drop, True, True, 0)
            # Dropdown ids
            self.id_drop = Gtk.ComboBoxText()
            for option in self.id_options:
                self.id_drop.append_text(option)
            self.id_drop.set_active(0)
            self.id_drop.connect("changed", self.id_changed)

            self.box_r.pack_start(self.id_drop, True, True, 0)
            # Positions print
            self.positions_label = Gtk.Label(label="")

            self.box_r.pack_start(self.positions_label, True, True, 0)
            # Buttons
            self.box_buttons = Gtk.Box(spacing=6)

            # Button Delete
            self.btn_delete = Gtk.Button(label="Delete")
            self.btn_delete.connect("clicked", self.btn_delete_clicked)

            self.box_buttons.pack_start(self.btn_delete, True, True, 0)
            # Button Next
            self.btn_next = Gtk.Button(label="Next")
            self.btn_next.connect("clicked", self.btn_next_clicked)

            self.box_buttons.pack_start(self.btn_next, True, True, 0)

            self.box_r.pack_start(self.box_buttons, True, True, 0)
            self.box.pack_start(self.box_r, True, True, 0)
        else:
            self.add(Gtk.Label(label='All images were already marked'))
            
    def btn_next_clicked(self, widget):
        curr_local = self.local_drop.get_model()[self.local_drop.get_active()][0]
        #while onlyfiles[self.img_iter] in data:
        self.img_iter +=1
        #    if self.img_iter >= len(onlyfiles):
#                break
        if self.img_iter < len(onlyfiles):
            self.img_name = onlyfiles[self.img_iter]
            pix = GdkPixbuf.Pixbuf.new_from_file(path+self.img_name)
            self.img_height = pix.get_height()
            self.img_width = pix.get_width()
            pixbuf = GdkPixbuf.Pixbuf.new_from_file_at_scale(path+self.img_name, self.width, -1, True)
            self.height = pixbuf.get_height()
            self.image.set_from_pixbuf(pixbuf)
            #data[self.img_name] = {"tags": preset["tags"],"position" : {}, "technic" : preset["technic"], "place": curr_local}
            data[self.img_name]["place"] = curr_local
            self.props.title = self.img_name  + ' ' + str(self.img_iter+1) + '/' + str(len(onlyfiles))
            self.update_positions()
        else:
            self.close()
    def btn_delete_clicked(self, widget):
        if os.path.exists(path+self.img_name):
            os.remove(path+self.img_name)
        else:
            print("The file does not exist")
            exit()
        del(onlyfiles[self.img_iter])
        del(data[self.img_name])
        if self.img_iter < len(onlyfiles):
            while onlyfiles[self.img_iter] in data:
                self.img_iter +=1
                if self.img_iter >= len(onlyfiles):
                    break
            if self.img_iter < len(onlyfiles):
                self.img_name = onlyfiles[self.img_iter]
                pix = GdkPixbuf.Pixbuf.new_from_file(path+self.img_name)
                self.img_height = pix.get_height()
                self.img_width = pix.get_width()
                pixbuf = GdkPixbuf.Pixbuf.new_from_file_at_scale(path+self.img_name, self.width, -1, True)
                self.height = pixbuf.get_height()
                self.image.set_from_pixbuf(pixbuf)
                data[self.img_name] = {"tags": preset["tags"], "position" : {}, "technic" : preset["technic"], "place": self.place_options[0]}
                self.positions_label.props.label =  "File Deleted!"
                self.props.title = self.img_name  + ' ' + str(self.img_iter+1) + '/' + str(len(onlyfiles))
            else:
                self.close()
        else:
            self.close()
    def id_changed(self, combo):
        self.update_positions()
    def local_changed(self, combo):
        tree_iter = combo.get_active_iter()
        if tree_iter is not None:
            model = combo.get_model()
            curr_local = model[tree_iter][0]
            data[self.img_name]["place"] = curr_local
    def onclick_position (self,box , event):
        curr_id = self.id_drop.get_model()[self.id_drop.get_active()][0]
        x = int((event.x/self.width)*self.img_width)
        y = int((event.y/self.height)*self.img_height)
        
        if curr_id in data[self.img_name]['position']:
            data[self.img_name]['position'][curr_id].append([x,y])
        else:
            data[self.img_name]['position'][curr_id] = [[x,y]]
        self.update_positions()
    def update_positions(self):
        label_text = ""
        curr_id = self.id_drop.get_model()[self.id_drop.get_active()][0]
        if curr_id in data[self.img_name]['position']:
            for pos in data[self.img_name]['position'][curr_id]:
                label_text+='['+str(pos[0])+', '+str(pos[1])+']\n'
        self.positions_label.props.label =  label_text

try:
    with open('../algorithm_performance_data/real_positions/colors_v2.json', 'r') as outfile:
        data = json.load(outfile)
except:
    data = {}
        
preset = {"tags": ["v2"],
          "technic" : "colors",
          "id_options" : ["blue", "red", "green", "yellow"], 
          "place_options" : ["daylight", "night right bulb", "night center bulb", "night left bulb"]}
path = '../img/colors/v2/'
onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
onlyfiles.sort()

img_name = ''
win = im_wiz_win(preset)
win.connect("destroy", Gtk.main_quit)
win.show_all()
Gtk.main()

with open('../algorithm_performance_data/real_positions/colors_v2.json', 'w') as outfile:
    json.dump(data, outfile)

KeyError: 'IMG_20201229_111104611.jpg'

KeyError: 'IMG_20201229_111104611.jpg'

KeyError: 'IMG_20201229_111104611.jpg'

KeyError: 'IMG_20201229_111104611.jpg'

KeyError: 'IMG_20201229_111104611.jpg'

KeyError: 'IMG_20201229_111104611.jpg'

KeyError: 'IMG_20201229_111104611.jpg'

In [64]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

# little correction...

path = '../img/colors/'

onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
onlyfiles.sort()

for im_name in onlyfiles:
    im = cv2.imread(path+im_name)
    im[2900:3119, 4059:4159,:] =  im[2900:3119, 4059-100:4159-100,:]
    cv2.imwrite(path+im_name, im)